In [ ]:
# tmm gse dataloader

In [3]:
import os
import torch
from inria_dataloader import get_inria_dataloader
from gse import GradientSelfEnsemble
from tmm import TransformerMaskingMatrix

def test_full_pipeline():
    data_root = "/opt/data/private/BlackBox/data/INRIAPerson/"
    detr_model_dir = "/opt/data/private/BlackBox/detr"
    batch_size = 2
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"使用设备：{device}")

    # 测试数据加载
    try:
        dataloader = get_inria_dataloader(
            data_root=data_root,
            split="Test",
            batch_size=batch_size,
            num_workers=0
        )
        print(f"✅ 数据加载器创建成功，测试集样本数：{len(dataloader.dataset)}")
        img_batch, _ = next(iter(dataloader))
        assert img_batch.shape == (batch_size, 3, 640, 640), \
            f"数据格式错误，预期{(batch_size,3,640,640)}，实际{img_batch.shape}"
        print(f"✅ 数据格式验证通过：{img_batch.shape}")
    except Exception as e:
        print(f"❌ 数据加载失败：{e}")
        return

    # 测试DETR模型加载及输出类别数
    try:
        model = torch.hub.load(
            detr_model_dir,
            "detr_resnet50",
            pretrained=True,
            source="local"
        )
        model.to(device)
        model.eval()
        # 验证模型原始输出类别数
        with torch.no_grad():
            img_batch_cuda = img_batch.to(device)
            orig_outputs = model(img_batch_cuda)
            num_classes = orig_outputs['pred_logits'].shape[-1]
        print(f"✅ DETR模型加载成功，实际输出类别数：{num_classes}")
    except Exception as e:
        print(f"❌ DETR模型加载失败：{e}")
        return

    # 测试TMM模块
    try:
        tmm = TransformerMaskingMatrix(
            num_enc_layers=6,
            num_dec_layers=6,
            p_base=0.2
        )
        tmm.register_hooks(model)
        with torch.no_grad():
            outputs_tmm = model(img_batch_cuda)
        assert "pred_logits" in outputs_tmm, "TMM输出格式错误"
        tmm.remove_hooks()
        print("✅ TMM模块与模型协同成功")
    except Exception as e:
        print(f"❌ TMM模块测试失败：{e}")
        return

    # 测试GSE模块（适配实际类别数）
    try:
        gse = GradientSelfEnsemble(model)
        img_list = [img_batch_cuda[i] for i in range(batch_size)]
        with torch.no_grad():
            gse_logits = gse(img_list, return_all_layers=False)
        # 按模型实际类别数验证
        assert gse_logits.shape == (batch_size, 100, num_classes), \
            f"GSE输出格式错误，预期{(batch_size,100,num_classes)}，实际{gse_logits.shape}"
        print("✅ GSE模块与模型协同成功")
    except Exception as e:
        print(f"❌ GSE模块测试失败：{e}")
        return

    print("\n🎉 所有组件协同测试通过！")
    print("验证结论：inria_dataloader.py、gse.py、tmm.py、DETR模型与数据集完全兼容")

if __name__ == "__main__":
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    test_full_pipeline()

使用设备：cuda
✅ 数据加载器创建成功，测试集样本数：288
✅ 数据格式验证通过：torch.Size([2, 3, 640, 640])
✅ DETR模型加载成功，实际输出类别数：92
✅ TMM已移除所有hook
✅ TMM已注册24个hook（6 encoder + 6 decoder）
✅ 采样策略：categorical（符合论文设置）
✅ TMM已移除所有hook
✅ TMM模块与模型协同成功
✅ GSE模块与模型协同成功

🎉 所有组件协同测试通过！
验证结论：inria_dataloader.py、gse.py、tmm.py、DETR模型与数据集完全兼容
